In [4]:
#NOT THIS
import json

def transform_messages(conversation):
    """Replace movie IDs with actual movie names."""
    movie_mentions = conversation.get("movieMentions", {})
    transformed_messages = []
        
    for message in conversation["messages"]:
        text = message["text"]
        # Replace all movie IDs with their names, skipping None values
        for movie_id, movie_name in movie_mentions.items():
            if movie_name is not None:  # Only replace if we have a valid movie name
                text = text.replace(f"@{movie_id}", movie_name)
            else:
                # Keep the original ID if no name is available
                text = text.replace(f"@{movie_id}", f"movie_{movie_id}")
        
        transformed_message = {
            "text": text,
            "senderWorkerId": message["senderWorkerId"]
        }
        transformed_messages.append(transformed_message)
    
    return transformed_messages, conversation["initiatorWorkerId"]

def format_dialogue(messages, initiator_id):
    """Format messages into dialogue string."""
    dialogue = []
    for msg in messages:
        # Identify if sender is initiator or respondent
        sender_type = "User" if msg["senderWorkerId"] == initiator_id else "Recommender"
        dialogue.append(f"{sender_type}: {msg['text']}")
    
    # Return dialogue wrapped in curly braces
    return "{" + " | ".join(dialogue) + "}"

def process_file(input_path, output_path):
    """Process the entire file from original format to final dialogue format."""
    try:
        with open(input_path, 'r', encoding='utf-8') as f, open(output_path, 'w', encoding='utf-8') as out_f:
            line_count = 0
            error_count = 0
            for line in f:
                if line.strip():
                    try:
                        # Load original conversation
                        conversation = json.loads(line)
                        
                        # Transform movie IDs to names
                        transformed_messages, initiator_id = transform_messages(conversation)
                        
                        # Format into final dialogue
                        final_dialogue = format_dialogue(transformed_messages, initiator_id)
                        
                        # Write to output file
                        out_f.write(final_dialogue + '\n')
                        line_count += 1
                        
                        # Print progress every 1000 lines
                        if line_count % 1000 == 0:
                            print(f"Processed {line_count} conversations...")
                            
                    except Exception as e:
                        error_count += 1
                        print(f"Error processing conversation: {str(e)}")
                        continue
            
            print(f"Processing complete. Successfully processed {line_count} conversations.")
            if error_count > 0:
                print(f"Encountered {error_count} errors during processing.")
                
    except Exception as e:
        print(f"Error opening files: {str(e)}")
        raise

def main():
    # Define file paths for both files
    file_pairs = [
        {
            'input': '/home/Nema/UniCRS_GraphRAG/data/redial/test_data_raw.jsonl',
            'output': '/home/Nema/UniCRS_GraphRAG/data/redial/test_data.jsonl'
        },
        {
            'input': '/home/Nema/UniCRS_GraphRAG/data/redial/train_data_raw.jsonl',
            'output': '/home/Nema/UniCRS_GraphRAG/data/redial/train_data.jsonl'
        }
    ]
    
    # Process each file pair
    for file_pair in file_pairs:
        print(f"\nProcessing {file_pair['input']}...")
        process_file(file_pair['input'], file_pair['output'])
        
        # Print sample of final output
        print(f"\nSample of transformed conversation from {file_pair['output']}:")
        try:
            with open(file_pair['output'], 'r', encoding='utf-8') as f:
                print(f.readline().strip())
        except Exception as e:
            print(f"Error reading sample: {str(e)}")
        
        print(f"Transformation complete for {file_pair['input']}")

if __name__ == "__main__":
    main()


Processing /home/Nema/UniCRS_GraphRAG/data/redial/test_data_raw.jsonl...
Processed 1000 conversations...
Processing complete. Successfully processed 1342 conversations.

Sample of transformed conversation from /home/Nema/UniCRS_GraphRAG/data/redial/test_data.jsonl:
{User: Hi I am looking for a movie like Super Troopers (2001) | Recommender: You should watch Police Academy  (1984) | User: Is that a great one? I have never seen it. I have seen American Pie  | User: I mean American Pie  (1999) | Recommender: Yes Police Academy  (1984) is very funny and so is Police Academy 2: Their First Assignment (1985) | User: It sounds like I need to check them out | Recommender: yes you will enjoy them | User: I appreciate your time. I will need to check those out. Are there any others you would recommend? | Recommender: yes Lethal Weapon (1987) | User: Thank you i will watch that too | Recommender: and also Beverly Hills Cop (1984) | User: Thanks for the suggestions. | Recommender: you are welcome 

Processed 2000 conversations...
Processed 3000 conversations...
Processed 4000 conversations...
Processed 5000 conversations...
Processed 6000 conversations...
Processed 7000 conversations...
Processed 8000 conversations...
Processed 9000 conversations...
Processed 10000 conversations...
Processing complete. Successfully processed 10005 conversations.
Encountered 1 errors during processing.

Sample of transformed conversation from /home/Nema/UniCRS_GraphRAG/data/redial/train_data.jsonl:
{User: Hi there, how are you? I'm looking for movie recommendations | Recommender: I am doing okay. What kind of movies do you like? | User: I like animations like The Triplets of Belleville (2003) and Waking Life (2001) | User: I also enjoy Mary and Max (2009) | User: Anything artistic | Recommender: You might like The Boss Baby (2017) that was a good movie. | User: What's it about? | Recommender: It has Alec Baldwin it is about a baby that works for a company and gets adopted it is very funny | User: 

In [1]:
# NOT THIS
import json
import csv

def transform_messages(conversation):
    """Replace movie IDs with actual movie names."""
    movie_mentions = conversation.get("movieMentions", {})
    transformed_messages = []

    for message in conversation["messages"]:
        text = message["text"]
        # Replace all movie IDs with their names, skipping None values
        for movie_id, movie_name in movie_mentions.items():
            if movie_name is not None:  # Only replace if we have a valid movie name
                text = text.replace(f"@{movie_id}", movie_name)
            else:
                # Keep the original ID if no name is available
                text = text.replace(f"@{movie_id}", f"movie_{movie_id}")

        transformed_message = {
            "text": text,
            "senderWorkerId": message["senderWorkerId"]
        }
        transformed_messages.append(transformed_message)

    return transformed_messages, conversation["initiatorWorkerId"]

def format_dialogue(messages, initiator_id):
    """Format messages into dialogue string."""
    dialogue = []
    for msg in messages:
        # Identify if sender is initiator or respondent
        sender_type = "User" if msg["senderWorkerId"] == initiator_id else "Recommender"
        dialogue.append(f"{sender_type}: {msg['text']}")

    # Return dialogue as a single string
    return " | ".join(dialogue)

def process_file(input_path, output_path):
    """Process the file and output as CSV."""
    try:
        with open(input_path, 'r', encoding='utf-8') as f, open(output_path, 'w', encoding='utf-8', newline='') as out_f:
            csv_writer = csv.DictWriter(out_f, fieldnames=["conversation_id", "full_dialogue"])
            csv_writer.writeheader()

            line_count = 0
            error_count = 0

            for line in f:
                if line.strip():
                    try:
                        # Load original conversation
                        conversation = json.loads(line)

                        # Transform movie IDs to names
                        transformed_messages, initiator_id = transform_messages(conversation)

                        # Format into final dialogue
                        final_dialogue = format_dialogue(transformed_messages, initiator_id)

                        # Write to CSV
                        csv_writer.writerow({
                            "conversation_id": conversation.get("conversationId", "unknown"),
                            "full_dialogue": final_dialogue
                        })

                        line_count += 1

                        # Print progress every 1000 lines
                        if line_count % 1000 == 0:
                            print(f"Processed {line_count} conversations...")

                    except Exception as e:
                        error_count += 1
                        print(f"Error processing conversation: {str(e)}")
                        continue

            print(f"Processing complete. Successfully processed {line_count} conversations.")
            if error_count > 0:
                print(f"Encountered {error_count} errors during processing.")

    except Exception as e:
        print(f"Error opening files: {str(e)}")
        raise

def main():
    # Define file paths for both files
    file_pairs = [
        {
            'input': '/home/Nema/UniCRS_GraphRAG/data/redial/test_data_raw.jsonl',
            'output': '/home/Nema/UniCRS_GraphRAG/data/redial/test_data.csv'
        },
        {
            'input': '/home/Nema/UniCRS_GraphRAG/data/redial/train_data_raw.jsonl',
            'output': '/home/Nema/UniCRS_GraphRAG/data/redial/train_data.csv'
        }
    ]

    # Process each file pair
    for file_pair in file_pairs:
        print(f"\nProcessing {file_pair['input']}...")
        process_file(file_pair['input'], file_pair['output'])

        # Print sample of final output
        print(f"\nSample of transformed conversation from {file_pair['output']}:")
        try:
            with open(file_pair['output'], 'r', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                print(next(reader))
        except Exception as e:
            print(f"Error reading sample: {str(e)}")

        print(f"Transformation complete for {file_pair['input']}")

if __name__ == "__main__":
    main()



Processing /home/Nema/UniCRS_GraphRAG/data/redial/test_data_raw.jsonl...
Processed 1000 conversations...
Processing complete. Successfully processed 1342 conversations.

Sample of transformed conversation from /home/Nema/UniCRS_GraphRAG/data/redial/test_data.csv:
{'conversation_id': '20001', 'full_dialogue': 'User: Hi I am looking for a movie like Super Troopers (2001) | Recommender: You should watch Police Academy  (1984) | User: Is that a great one? I have never seen it. I have seen American Pie  | User: I mean American Pie  (1999) | Recommender: Yes Police Academy  (1984) is very funny and so is Police Academy 2: Their First Assignment (1985) | User: It sounds like I need to check them out | Recommender: yes you will enjoy them | User: I appreciate your time. I will need to check those out. Are there any others you would recommend? | Recommender: yes Lethal Weapon (1987) | User: Thank you i will watch that too | Recommender: and also Beverly Hills Cop (1984) | User: Thanks for the 

In [1]:
# THIS
import json

def transform_messages(conversation):
    """Replace movie IDs with actual movie names."""
    movie_mentions = conversation.get("movieMentions", {})
    transformed_messages = []
    
    for message in conversation["messages"]:
        text = message["text"]
        # Replace all movie IDs with their names, skipping None values
        for movie_id, movie_name in movie_mentions.items():
            if movie_name is not None:  # Only replace if we have a valid movie name
                text = text.replace(f"@{movie_id}", movie_name)
            else:
                # Keep the original ID if no name is available
                text = text.replace(f"@{movie_id}", f"movie_{movie_id}")
        
        transformed_message = {
            "text": text,
            "senderWorkerId": message["senderWorkerId"]
        }
        transformed_messages.append(transformed_message)
    
    return transformed_messages, conversation["initiatorWorkerId"]

def format_dialogue(messages, initiator_id):
    """Format messages into dialogue string."""
    dialogue = []
    for msg in messages:
        # Identify if sender is initiator or respondent
        sender_type = "User" if msg["senderWorkerId"] == initiator_id else "Recommender"
        dialogue.append(f"{sender_type}: {msg['text']}")
    
    # Return dialogue without curly braces
    return " | ".join(dialogue)

def process_file(input_path, output_path):
    """Process the entire file from original format to final dialogue format."""
    try:
        with open(input_path, 'r', encoding='utf-8') as f, open(output_path, 'w', encoding='utf-8') as out_f:
            line_count = 0
            error_count = 0
            for line in f:
                if line.strip():
                    try:
                        # Load original conversation
                        conversation = json.loads(line)
                        
                        # Transform movie IDs to names
                        transformed_messages, initiator_id = transform_messages(conversation)
                        
                        # Format into final dialogue
                        final_dialogue = format_dialogue(transformed_messages, initiator_id)
                        
                        # Write to output file
                        out_f.write(final_dialogue + '\n')
                        line_count += 1
                        
                        # Print progress every 1000 lines
                        if line_count % 1000 == 0:
                            print(f"Processed {line_count} conversations...")
                            
                    except Exception as e:
                        error_count += 1
                        print(f"Error processing conversation: {str(e)}")
                        continue
            
            print(f"Processing complete. Successfully processed {line_count} conversations.")
            if error_count > 0:
                print(f"Encountered {error_count} errors during processing.")
                
    except Exception as e:
        print(f"Error opening files: {str(e)}")
        raise

def main():
    # Define file paths for both files with .txt extension
    file_pairs = [
        {
            'input': '/home/Nema/UniCRS_GraphRAG/data/redial/test_data_raw.jsonl',
            'output': '/home/Nema/UniCRS_GraphRAG/data/redial/test_data.txt'
        },
        {
            'input': '/home/Nema/UniCRS_GraphRAG/data/redial/train_data_raw.jsonl',
            'output': '/home/Nema/UniCRS_GraphRAG/data/redial/train_data.txt'
        }
    ]
    
    # Process each file pair
    for file_pair in file_pairs:
        print(f"\nProcessing {file_pair['input']}...")
        process_file(file_pair['input'], file_pair['output'])
        
        # Print sample of final output
        print(f"\nSample of transformed conversation from {file_pair['output']}:")
        try:
            with open(file_pair['output'], 'r', encoding='utf-8') as f:
                print(f.readline().strip())
        except Exception as e:
            print(f"Error reading sample: {str(e)}")
        
        print(f"Transformation complete for {file_pair['input']}")

if __name__ == "__main__":
    main()


Processing /home/Nema/UniCRS_GraphRAG/data/redial/test_data_raw.jsonl...
Processed 1000 conversations...
Processing complete. Successfully processed 1342 conversations.

Sample of transformed conversation from /home/Nema/UniCRS_GraphRAG/data/redial/test_data.txt:
User: Hi I am looking for a movie like Super Troopers (2001) | Recommender: You should watch Police Academy  (1984) | User: Is that a great one? I have never seen it. I have seen American Pie  | User: I mean American Pie  (1999) | Recommender: Yes Police Academy  (1984) is very funny and so is Police Academy 2: Their First Assignment (1985) | User: It sounds like I need to check them out | Recommender: yes you will enjoy them | User: I appreciate your time. I will need to check those out. Are there any others you would recommend? | Recommender: yes Lethal Weapon (1987) | User: Thank you i will watch that too | Recommender: and also Beverly Hills Cop (1984) | User: Thanks for the suggestions. | Recommender: you are welcome | R

In [2]:
import statistics

def count_tokens(text):
    """Simple whitespace-based tokenization."""
    return len(text.split())

def analyze_file(file_path):
    """Read file and analyze token counts."""
    token_counts = []
    total_tokens = 0
    line_count = 0
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.strip():
                    tokens = count_tokens(line.strip())
                    token_counts.append(tokens)
                    total_tokens += tokens
                    line_count += 1
                    
                    # Print progress every 1000 lines
                    if line_count % 1000 == 0:
                        print(f"Processed {line_count} lines...")
        
        # Calculate statistics
        avg_tokens = total_tokens / line_count if line_count > 0 else 0
        median_tokens = statistics.median(token_counts) if token_counts else 0
        
        # Print results
        print("\nToken Statistics:")
        print(f"Total lines processed: {line_count}")
        print(f"Total tokens: {total_tokens}")
        print(f"Average tokens per line: {avg_tokens:.2f}")
        print(f"Median tokens per line: {median_tokens}")
        print(f"Minimum tokens in a line: {min(token_counts)}")
        print(f"Maximum tokens in a line: {max(token_counts)}")
        
    except Exception as e:
        print(f"Error processing file: {str(e)}")

def main():
    # Define file paths
    files = [
        '/home/Nema/UniCRS_GraphRAG/data/redial/test_data.txt',
        '/home/Nema/UniCRS_GraphRAG/data/redial/train_data.txt'
    ]
    
    # Process each file
    for file_path in files:
        print(f"\nAnalyzing {file_path}...")
        analyze_file(file_path)

if __name__ == "__main__":
    main()


Analyzing /home/Nema/UniCRS_GraphRAG/data/redial/test_data.txt...
Processed 1000 lines...

Token Statistics:
Total lines processed: 1356
Total tokens: 218651
Average tokens per line: 161.25
Median tokens per line: 150.0
Minimum tokens in a line: 1
Maximum tokens in a line: 465

Analyzing /home/Nema/UniCRS_GraphRAG/data/redial/train_data.txt...
Processed 1000 lines...
Processed 2000 lines...
Processed 3000 lines...
Processed 4000 lines...
Processed 5000 lines...
Processed 6000 lines...
Processed 7000 lines...
Processed 8000 lines...
Processed 9000 lines...
Processed 10000 lines...

Token Statistics:
Total lines processed: 10125
Total tokens: 1738489
Average tokens per line: 171.70
Median tokens per line: 163
Minimum tokens in a line: 1
Maximum tokens in a line: 1189
